# Homework 1: A Simple Sentiment Analysis of Tweets

The objective of this homework is to assess your current skills and your ability to solve an unknown problem. To successfully complete this homework, you may use any resources available to you.

You need to accomplish the following tasks:
1. Choose a Twitter API.
1. Configure access to the Twitter API
2. Identify a trending topic on Twitter.
3. Get at least 500 tweets on your trending topic.
3. Find lists of stopwords, positive words, and negative words.
4. Calculate the ratio of positive to negative words in your sample.
    
Answer the following questions: 
* __What is the ratio of positive to negative words on your trending topic?__ 
* __What is your interpretation of the ratio?__
* __What is the managerial insight that you could offer based on your results?__

If you use tutorials/code snippets that you find on the internet to complete this task, make sure that you reference them. Also make sure that the Jupyter notebook is free of mistakes, well-documented, and professionally formatted before you submit it.

This homework is due on Friday, 12 2018.

Title: Twitter Sentimental Analysis
Author: Xiao Zhang

Note: before reviewing the code, I take the "words" in the question as "topics"

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
import re
import tweepy
from textblob import TextBlob
from twitter import Twitter, OAuth

In [ ]:
class TwitterConsumer(object):
    
    try:
        import json
    except ImportError:
        import simplejson as json
    
    
    # Import necessary methods from "twitter" library
    from twitter import Twitter, OAuth
    
    try:
        
        # My credentials to access Twitter API
        consumer_key = 'E5NyOKATic2KVFQD8bFACi7ws'
        consumer_secret = 'dHHBhbfpbZNDPZlUDD4ZMWYYdSwpaVbq2tsLPCrHPvZ2pe9etF'
        access_token = '940798876210155521-sIOX6DRJ78iI6aYz8pPxNvTEDRHAfxk'
        access_token_secret = 'YcB0GVNSzCxMw2QMs2MRucdmI3yOEDSZDTaCMyrotIUM9'

        oauth = OAuth(ACCESS_TOKEN, ACCESS_SECRET, CONSUMER_KEY, CONSUMER_SECRET)

        api = twitter.Api(  consumer_key='consumer_key',
                                    consumer_secret='consumer_secret',
                                    access_token_key='access_token',
                                    access_token_secret='access_token_secret')
    except:
            print("Error: Authentication Failed")
    
    # Remove links, special characters using simple regex statements
    # Reference link:
    # https://www.geeksforgeeks.org/twitter-sentiment-analysis-using-python/
    def revised_tweet(oauth, tweet):
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z])|(\w+:\
    S+)", " ", tweet).split())
    
    
    # Use textblob's sentiment method to classify sentiment of tweet
    def tweet_sentiment(oauth, tweet):
        # create TextBlob object of tweet text
        analysis = TextBlob(oauth.revised_tweet(tweet))
        # set sentiment
        if analysis.sentiment.polarity > 0:
            return 'positive words'
        elif analysis.sentiment.polarity == 0:
            return 'stopwords'
        else:
            return 'negative words'
    
    # Set up a null list and append 500 Twitter in 5 pages
    def get_raw_tweets(searchstr):
        resultpage = []
        for i in (1,5):
            searchresult = api.GetSearch(searchstr, per_page=100, page=i)
            resultpage.append(searchresult)
        return resultpage
    
    # Apply search api
    def get_tweets(query, count = 10):
        # Initiate the connection to Twitter REST API
        twitter = Twitter(auth=oauth)
        # Search for latest tweets about "query topic"
        tweets= twitter.search.resultpage(q=query)
        
        try:
            
            for tweet in tweets:
                # Set up the dict of text and sentiment
                new_tweet = {}
                # saving key as 'text' of tweet
                new_tweet['text'] = tweet.text
                # saving key asy 'sentiment' of tweet
                new_tweet['sentiment'] = tweet_sentiment(tweet.text)

                # appending tweet to tweets list

                if tweet.retweet_count > 0:
                    # if tweet has retweets, ensure that it is appended only once
                    if new_tweet not in tweets:
                        tweets.append(new_tweet)
                else:
                    tweets.append(new_tweet)

              # return parsed tweets 
            return new_tweet
            
        except twitter.TwitterError as e:
                # print error (if any)
                print("Error : " + str(e))

         

def main():
    # creating object c1 of TwitterClient Class
    c1 = TwitterConsumer()
    # Get tweets about the past CES event
    tweets = c1.get_tweets('#CES', 500)
    
    # Find positive tweets from tweets
    ptweets = [tweet for tweet in tweets if tweet['sentiment'] == 'positive']
    # percentage of positive tweets
    print("Positive tweets ratio: {} %".format(100*len(ptweets)/len(tweets)))
    # Find negative tweets from tweets
    ntweets = [tweet for tweet in tweets if tweet['sentiment'] == 'negative']
    # percentage of negative tweets
    print("Negative tweets ratio: {} %".format(100*len(ntweets)/len(tweets)))
    # Find stopwords from tweets
    ntweets = [tweet for tweet in tweets if tweet['sentiment'] == 'neutral']
    
    
if __name__ == "__main__":
    # calling main function
    main()